In [ ]:
from openai import OpenAI
import os
from getpass import getpass
import base64
from io import BytesIO
import json

from typing import List
from pydantic import BaseModel, confloat

In [ ]:
# input api key
# get from https://openai.com/api/
api_key = getpass("Enter your OpenAI API key: ")

#gpt api
gpt_client = OpenAI(api_key=api_key)

In [ ]:
# convert image to base64 to pass into openai api
def image_to_base64(img):
    """Convert a PIL Image to a base64 data URI for OpenAI API."""
    buffer = BytesIO()
    img.save(buffer, format="JPEG")  # or "PNG" if you prefer
    b64 = base64.b64encode(buffer.getvalue()).decode("utf-8")
    return f"data:image/jpeg;base64,{b64}"

In [ ]:
# huskyeats api helpers
import requests

def get_menu_items(hallid, meal, date):
    url = "https://husky-eats.onrender.com/api/menu"
    r = requests.get(url, params={"hallid": hallid, "meal": meal, "date": date})
    r.raise_for_status()
    return [
        {"name": item["name"], "id": item["id"]}
        for item in r.json()
        if "name" in item and "id" in item
    ]

In [ ]:
# Pydantic Output Classes
class ClassifiedItem(BaseModel):
    id: int
    name: str
    confidence: confloat(ge=0.0, le=1.0)

class ClassificationResult(BaseModel):
    items: List[ClassifiedItem]   # [{id,name,confidence}, ...]
    explanation: str

In [ ]:
# gpt model 1, item classification
def gpt_item_classification(pil_img, client, model="gpt-5-mini"):
    # convert PIL image to base64
    image_url = image_to_base64(pil_img)

    # pair items with scores (top-N already ranked upstream)
    items_payload = json.dumps(get_menu_items())

    # prompt
    prompt_text = (
        "You are a food identification expert specializing in dining hall meals.\n\n"
        "You will be shown an image of a plate and a list of candidate menu items \n"
        "Your task: determine which of these items are actually present on the plate.\n\n"
        "Guidelines:\n"
        "- Use both the image and the ranking scores; scores are hints, not ground truth.\n"
        "- Focus only on visible foods; ignore background objects like trays or utensils.\n"
        "- Only choose from the provided menu item list — do not invent new items.\n\n"
        "Output Format (strict JSON):\n"
        '- \"items\": an array of objects, each with fields {\"id\", \"name\", \"confidence\"}\n'
        '- \"confidence\": a float in [0,1] for each chosen item\n'
        '- \"explanation\": a brief 1–2 sentence rationale describing your reasoning.\n'
    )

    # call the gpt responses API
    response = client.responses.parse(
        model=model,
        input=[
            {
                "role": "user",
                "content": [
                    { "type": "input_text", "text": prompt_text },
                    { "type": "input_text", "text": items_payload },
                    { "type": "input_image", "image_url": image_url }
                ],
            }
        ],
        text_format=ClassificationResult
    )

    return response.output_parsed.model_dump()

In [ ]:
# output
classification_result = gpt_item_classification(pil_image, items_ranked, gpt_client)
print(classification_result)